In [10]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json
import time
import math
# from tensorflow import keras
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, r2_score
from keras.models import Sequential
from keras.layers import Dense, Dropout,Input
from pathlib import Path
from keras.layers import LSTM
from keras.layers import Activation 
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Flatten, Conv1D, SpatialDropout1D
# from keras.layers import Embedding,Conv1D,LSTM,Input,TimeDistributed,,Flatten,Dropout
from tensorflow.keras.optimizers import Adam
from sklearn import preprocessing
from tensorflow.keras.utils import to_categorical
from keras.metrics import categorical_crossentropy
import glob
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm, datasets
from sklearn.metrics import f1_score

In [30]:
def get_data(c):
#         df = pd.read_csv("D:\\Projects\Federated Learning\\Datasets\\Fed Dataset Thermal (Transfer Learning Paper)\\"+self.country+".csv")
#         df = pd.read_csv('../Data/Correlation Based Data/'+c+'.csv')
        df = pd.read_csv('../Data/Synthetic/'+c+'.csv')
        y = df['label']
        drop_list = ['Unnamed: 0','Publication (Citation)','Data contributor','Heating strategy_building level',
                     'Year','Koppen climate classification','Climate','Building type','Database',
                     'City','Country','Outdoor monthly air temperature (¡C)','Thermal preference','Season','Cooling startegy_building level',
                     'Air movement preference','Humidity preference','Thermal comfort','PMV','Thermal sensation','label']
        drop = list(set(df.columns).intersection(drop_list))
        X = df.drop(drop, axis = 1)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
        
        y_train=to_categorical(y_train,num_classes=5)
        y_test=to_categorical(y_test,num_classes=5)
        
        X_train = np.asarray(X_train).astype(np.float32)
        y_train = np.asarray(y_train).astype(np.float32)
        
        #for LSTM CNN model
        X_train=np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],-1)
        X_test=np.array(X_test).reshape(X_test.shape[0],X_test.shape[1],-1)
        #For DNN Model
        #y_train = np.reshape(y_train, (len(y_train), len(y_train[0])))
        
        return X_train, X_test, y_train, y_test
    

In [24]:
# labels = ['India', 'Italy', 'Pakistan', 'Philippines', 'Portugal', 'Singapore', 'Sweden', 'Thailand', 'Tunisia','UK',"Scales","Medium_US"]
# path = 'countries results/'

path = 'Synthetic Data Results/'
labels = ['syn_data_0','syn_data_1',"syn_data_2","syn_data_3","syn_data_4", "syn_data_5","syn_data_6","syn_data_7","syn_data_8","syn_data_9"]

In [25]:
def build_model(X_train):
      MLP_model=Sequential()
      MLP_model.add(Dense(1024,activation='relu',input_shape = (len(list(X_train[0])),)))
#       MLP_model.add(Dense(512,activation='relu',kernel_initializer='glorot_uniform'))
#       MLP_model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform'))
#       MLP_model.add(Dense(128,activation='relu',kernel_initializer='glorot_uniform'))
#       MLP_model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform'))
#       MLP_model.add(Dense(32,activation='relu',kernel_initializer='glorot_uniform'))
#       MLP_model.add(Dense(16,activation='relu',kernel_initializer='glorot_uniform'))
#       MLP_model.add(Dense(8,activation='relu',kernel_initializer='glorot_uniform'))
      MLP_model.add(Dense(5,activation='softmax'))
      MLP_model.compile(optimizer=Adam(lr=0.001),metrics=['accuracy'],loss='categorical_crossentropy',weighted_metrics=['accuracy'])
      return MLP_model

In [26]:
def get_thermal_model(X_train):
        #X_train=np.array(X_train).reshape(X_train.shape[0],X_train.shape[1],-1)
        model=Sequential()
        #print(self.X_train.shape[2],"----------------")
        #model.add(Embedding(101,256,input_length=8,))
        model.add(Conv1D(filters=128,kernel_size=5,padding='same',input_shape=(X_train.shape[1],X_train.shape[2])))
        model.add(LSTM(256,return_sequences=True))
        model.add(LSTM(256,return_sequences=True))
        model.add(Flatten())
        model.add(Dense(64,activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(16,activation='relu'))
        model.add(Dropout(0.1))
        model.add(Dense(5,activation='softmax'))
        model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.001,beta_1=0.99,beta_2=0.999),metrics=['accuracy'],weighted_metrics=['accuracy'])
        return model

In [27]:
def save_history_full(c,performance):
        df = pd.DataFrame(performance)
        df.to_csv(path+c+'_LSTMCNN_performance.csv', mode='w',header=True)

In [28]:
def my_to_catagorical( y_predict):
        updated_y_predict = []
        for p in y_predict:
            n = max(p)
            for i in range(0,len(p)):
               if n == p[i]:
                   p[i] = 1
               else:
                   p[i] = 0
            updated_y_predict.append(p)
        return updated_y_predict

In [29]:
precision_all = []
for c in labels:
    X_train, X_test, y_train, y_test = get_data(c)
    model = build_model(X_train)
    history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=100)
    #save_history_full(c,history.history)
    train_p = model.predict(X_train)
    test_p = model.predict(X_test)
    temp_pre = []
    
    cat_train_p = my_to_catagorical(train_p)
    report = classification_report(y_train, cat_train_p,output_dict=True)
    precision = [[c,p['precision']] for [c,p] in report.items()]
    train_classwise_precision = precision[0:5]
    train_avg_precision = precision[8]
    
    temp_pre.append(train_avg_precision[1])
    temp_pre.extend([train_classwise_precision[0][1],train_classwise_precision[1][1],train_classwise_precision[2][1],train_classwise_precision[3][1],train_classwise_precision[4][1]])
    
    cat_test_p = my_to_catagorical(test_p)
    report = classification_report(y_test, cat_test_p,output_dict=True)
    precision = [[c,p['precision']] for [c,p] in report.items()]
    test_classwise_precision = precision[0:5]
    test_avg_precision = precision[8]
    
    temp_pre.append(test_avg_precision[1])
    temp_pre.extend([test_classwise_precision[0][1],test_classwise_precision[1][1],test_classwise_precision[2][1],test_classwise_precision[3][1],test_classwise_precision[4][1]])
    precision_all.append(temp_pre)

    

Epoch 1/100


C:\Users\user\anaconda3\envs\flaskproject\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


66/66 [==============================] - 0s 4ms/step - loss: 0.7325 - accuracy: 0.7205 - weighted_accuracy: 0.7205 - val_loss: 0.6649 - val_accuracy: 0.7533 - val_weighted_accuracy: 0.7533
Epoch 2/100
66/66 [==============================] - 0s 2ms/step - loss: 0.6296 - accuracy: 0.7548 - weighted_accuracy: 0.7548 - val_loss: 0.6825 - val_accuracy: 0.7456 - val_weighted_accuracy: 0.7456
Epoch 3/100
66/66 [==============================] - 0s 2ms/step - loss: 0.6248 - accuracy: 0.7562 - weighted_accuracy: 0.7562 - val_loss: 0.6582 - val_accuracy: 0.7511 - val_weighted_accuracy: 0.7511
Epoch 4/100
66/66 [==============================] - 0s 2ms/step - loss: 0.6016 - accuracy: 0.7619 - weighted_accuracy: 0.7619 - val_loss: 0.6872 - val_accuracy: 0.7489 - val_weighted_accuracy: 0.7489
Epoch 5/100
66/66 [==============================] - 0s 2ms/step - loss: 0.5910 - accuracy: 0.7652 - weighted_accuracy: 0.7652 - val_loss: 0.6749 - val_accuracy: 0.7489 - val_weighted_accuracy: 0.7489
Epoch 6

Epoch 42/100
66/66 [==============================] - 0s 2ms/step - loss: 0.4071 - accuracy: 0.8386 - weighted_accuracy: 0.8386 - val_loss: 0.7019 - val_accuracy: 0.7311 - val_weighted_accuracy: 0.7311
Epoch 43/100
66/66 [==============================] - 0s 2ms/step - loss: 0.4140 - accuracy: 0.8429 - weighted_accuracy: 0.8429 - val_loss: 0.7604 - val_accuracy: 0.7367 - val_weighted_accuracy: 0.7367
Epoch 44/100
66/66 [==============================] - 0s 2ms/step - loss: 0.3962 - accuracy: 0.8429 - weighted_accuracy: 0.8429 - val_loss: 0.6988 - val_accuracy: 0.7444 - val_weighted_accuracy: 0.7444
Epoch 45/100
66/66 [==============================] - 0s 2ms/step - loss: 0.3857 - accuracy: 0.8500 - weighted_accuracy: 0.8500 - val_loss: 0.6990 - val_accuracy: 0.7478 - val_weighted_accuracy: 0.7478
Epoch 46/100
66/66 [==============================] - 0s 3ms/step - loss: 0.3944 - accuracy: 0.8443 - weighted_accuracy: 0.8443 - val_loss: 0.6966 - val_accuracy: 0.7500 - val_weighted_accurac

66/66 [==============================] - 0s 2ms/step - loss: 0.2576 - accuracy: 0.9024 - weighted_accuracy: 0.9024 - val_loss: 0.8952 - val_accuracy: 0.7211 - val_weighted_accuracy: 0.7211
Epoch 83/100
66/66 [==============================] - 0s 2ms/step - loss: 0.2687 - accuracy: 0.8933 - weighted_accuracy: 0.8933 - val_loss: 0.7968 - val_accuracy: 0.7367 - val_weighted_accuracy: 0.7367
Epoch 84/100
66/66 [==============================] - 0s 2ms/step - loss: 0.2494 - accuracy: 0.9033 - weighted_accuracy: 0.9033 - val_loss: 0.8211 - val_accuracy: 0.7333 - val_weighted_accuracy: 0.7333
Epoch 85/100
66/66 [==============================] - 0s 2ms/step - loss: 0.2570 - accuracy: 0.9071 - weighted_accuracy: 0.9071 - val_loss: 0.8341 - val_accuracy: 0.7200 - val_weighted_accuracy: 0.7200
Epoch 86/100
66/66 [==============================] - 0s 2ms/step - loss: 0.2492 - accuracy: 0.9067 - weighted_accuracy: 0.9067 - val_loss: 0.8450 - val_accuracy: 0.7300 - val_weighted_accuracy: 0.7300
Epo

...

114/114 [==============================] - 0s 2ms/step - loss: 0.1058 - accuracy: 0.9695 - weighted_accuracy: 0.9695 - val_loss: 1.1854 - val_accuracy: 0.7288 - val_weighted_accuracy: 0.7288
Epoch 82/100
114/114 [==============================] - 0s 2ms/step - loss: 0.0968 - accuracy: 0.9734 - weighted_accuracy: 0.9734 - val_loss: 1.1635 - val_accuracy: 0.7141 - val_weighted_accuracy: 0.7141
Epoch 83/100
114/114 [==============================] - 0s 2ms/step - loss: 0.0870 - accuracy: 0.9788 - weighted_accuracy: 0.9788 - val_loss: 1.2152 - val_accuracy: 0.7359 - val_weighted_accuracy: 0.7359
Epoch 84/100
114/114 [==============================] - 0s 2ms/step - loss: 0.1036 - accuracy: 0.9676 - weighted_accuracy: 0.9676 - val_loss: 1.2125 - val_accuracy: 0.7083 - val_weighted_accuracy: 0.7083
Epoch 85/100
114/114 [==============================] - 0s 2ms/step - loss: 0.0896 - accuracy: 0.9780 - weighted_accuracy: 0.9780 - val_loss: 1.2657 - val_accuracy: 0.7224 - val_weighted_accuracy: 

C:\Users\user\anaconda3\envs\flaskproject\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


105/105 [==============================] - 0s 3ms/step - loss: 0.7380 - accuracy: 0.7190 - weighted_accuracy: 0.7190 - val_loss: 0.6821 - val_accuracy: 0.7437 - val_weighted_accuracy: 0.7437
Epoch 2/100
105/105 [==============================] - 0s 2ms/step - loss: 0.6582 - accuracy: 0.7494 - weighted_accuracy: 0.7494 - val_loss: 0.6838 - val_accuracy: 0.7486 - val_weighted_accuracy: 0.7486
Epoch 3/100
105/105 [==============================] - 0s 2ms/step - loss: 0.6271 - accuracy: 0.7619 - weighted_accuracy: 0.7619 - val_loss: 0.6900 - val_accuracy: 0.7319 - val_weighted_accuracy: 0.7319
Epoch 4/100
105/105 [==============================] - 0s 2ms/step - loss: 0.6190 - accuracy: 0.7598 - weighted_accuracy: 0.7598 - val_loss: 0.6772 - val_accuracy: 0.7375 - val_weighted_accuracy: 0.7375
Epoch 5/100
105/105 [==============================] - 0s 2ms/step - loss: 0.6104 - accuracy: 0.7616 - weighted_accuracy: 0.7616 - val_loss: 0.7237 - val_accuracy: 0.7236 - val_weighted_accuracy: 0.72

105/105 [==============================] - 0s 2ms/step - loss: 0.4223 - accuracy: 0.8339 - weighted_accuracy: 0.8339 - val_loss: 0.8094 - val_accuracy: 0.7243 - val_weighted_accuracy: 0.7243
Epoch 42/100
105/105 [==============================] - 0s 2ms/step - loss: 0.4115 - accuracy: 0.8381 - weighted_accuracy: 0.8381 - val_loss: 0.7939 - val_accuracy: 0.7257 - val_weighted_accuracy: 0.7257
Epoch 43/100
105/105 [==============================] - 0s 2ms/step - loss: 0.3985 - accuracy: 0.8423 - weighted_accuracy: 0.8423 - val_loss: 0.8082 - val_accuracy: 0.7264 - val_weighted_accuracy: 0.7264
Epoch 44/100
105/105 [==============================] - 0s 2ms/step - loss: 0.4092 - accuracy: 0.8375 - weighted_accuracy: 0.8375 - val_loss: 0.7960 - val_accuracy: 0.7306 - val_weighted_accuracy: 0.7306
Epoch 45/100
105/105 [==============================] - 0s 2ms/step - loss: 0.3929 - accuracy: 0.8446 - weighted_accuracy: 0.8446 - val_loss: 0.7926 - val_accuracy: 0.7306 - val_weighted_accuracy: 

105/105 [==============================] - 0s 2ms/step - loss: 0.2489 - accuracy: 0.9057 - weighted_accuracy: 0.9057 - val_loss: 0.9910 - val_accuracy: 0.7250 - val_weighted_accuracy: 0.7250
Epoch 82/100
105/105 [==============================] - 0s 2ms/step - loss: 0.2446 - accuracy: 0.9054 - weighted_accuracy: 0.9054 - val_loss: 0.9887 - val_accuracy: 0.7153 - val_weighted_accuracy: 0.7153
Epoch 83/100
105/105 [==============================] - 0s 2ms/step - loss: 0.2461 - accuracy: 0.9089 - weighted_accuracy: 0.9089 - val_loss: 1.0082 - val_accuracy: 0.7201 - val_weighted_accuracy: 0.7201
Epoch 84/100
105/105 [==============================] - 0s 2ms/step - loss: 0.2323 - accuracy: 0.9149 - weighted_accuracy: 0.9149 - val_loss: 0.9984 - val_accuracy: 0.7111 - val_weighted_accuracy: 0.7111
Epoch 85/100
105/105 [==============================] - 0s 2ms/step - loss: 0.2468 - accuracy: 0.9068 - weighted_accuracy: 0.9068 - val_loss: 1.0388 - val_accuracy: 0.7118 - val_weighted_accuracy: 

In [31]:
df_precision = pd.DataFrame(precision_all,columns =['tr_avg_pre','TrP1', 'TrP2','TrP3','TrP4','TrP5','ts_avg_pre','TsP1','TsP2','TsP3','TsP4','TsP5'] )
df_precision.to_csv(path+'precision_DNN.csv')

In [32]:
precision_all = []
for c in labels:
    X_train, X_test, y_train, y_test = get_data(c)
    model = get_thermal_model(X_train)
    history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=100)
#     save_history_full(c,history.history)
    train_p = model.predict(X_train)
    test_p = model.predict(X_test)
    temp_pre = []
    
    cat_train_p = my_to_catagorical(train_p)
    report = classification_report(y_train, cat_train_p,output_dict=True)
    precision = [[c,p['precision']] for [c,p] in report.items()]
    train_classwise_precision = precision[0:5]
    train_avg_precision = precision[8]
    
    temp_pre.append(train_avg_precision[1])
    temp_pre.extend([train_classwise_precision[0][1],train_classwise_precision[1][1],train_classwise_precision[2][1],train_classwise_precision[3][1],train_classwise_precision[4][1]])
    
    cat_test_p = my_to_catagorical(test_p)
    report = classification_report(y_test, cat_test_p,output_dict=True)
    precision = [[c,p['precision']] for [c,p] in report.items()]
    test_classwise_precision = precision[0:5]
    test_avg_precision = precision[8]
    
    temp_pre.append(test_avg_precision[1])
    temp_pre.extend([test_classwise_precision[0][1],test_classwise_precision[1][1],test_classwise_precision[2][1],test_classwise_precision[3][1],test_classwise_precision[4][1]])
    precision_all.append(temp_pre)


Epoch 1/100
66/66 [==============================] - 3s 15ms/step - loss: 1.0596 - accuracy: 0.5914 - weighted_accuracy: 0.5914 - val_loss: 0.8408 - val_accuracy: 0.6878 - val_weighted_accuracy: 0.6878
Epoch 2/100
66/66 [==============================] - 0s 6ms/step - loss: 0.7869 - accuracy: 0.7067 - weighted_accuracy: 0.7067 - val_loss: 0.7603 - val_accuracy: 0.7111 - val_weighted_accuracy: 0.7111
Epoch 3/100
66/66 [==============================] - 0s 6ms/step - loss: 0.7084 - accuracy: 0.7290 - weighted_accuracy: 0.7290 - val_loss: 0.6911 - val_accuracy: 0.7433 - val_weighted_accuracy: 0.7433
Epoch 4/100
66/66 [==============================] - 0s 6ms/step - loss: 0.6514 - accuracy: 0.7681 - weighted_accuracy: 0.7681 - val_loss: 0.6570 - val_accuracy: 0.7522 - val_weighted_accuracy: 0.7522
Epoch 5/100
66/66 [==============================] - 0s 7ms/step - loss: 0.6280 - accuracy: 0.7671 - weighted_accuracy: 0.7671 - val_loss: 0.6504 - val_accuracy: 0.7500 - val_weighted_accuracy: 0

66/66 [==============================] - 0s 6ms/step - loss: 0.0684 - accuracy: 0.9781 - weighted_accuracy: 0.9781 - val_loss: 2.1530 - val_accuracy: 0.7089 - val_weighted_accuracy: 0.7089
Epoch 42/100
66/66 [==============================] - 0s 6ms/step - loss: 0.0719 - accuracy: 0.9781 - weighted_accuracy: 0.9781 - val_loss: 2.1031 - val_accuracy: 0.7167 - val_weighted_accuracy: 0.7167
Epoch 43/100
66/66 [==============================] - 0s 6ms/step - loss: 0.0488 - accuracy: 0.9843 - weighted_accuracy: 0.9843 - val_loss: 2.0193 - val_accuracy: 0.7200 - val_weighted_accuracy: 0.7200
Epoch 44/100
66/66 [==============================] - 0s 6ms/step - loss: 0.0451 - accuracy: 0.9871 - weighted_accuracy: 0.9871 - val_loss: 1.9531 - val_accuracy: 0.7189 - val_weighted_accuracy: 0.7189
Epoch 45/100
66/66 [==============================] - 0s 6ms/step - loss: 0.0277 - accuracy: 0.9919 - weighted_accuracy: 0.9919 - val_loss: 2.0276 - val_accuracy: 0.7167 - val_weighted_accuracy: 0.7167
Epo

Epoch 82/100
66/66 [==============================] - 0s 6ms/step - loss: 0.0124 - accuracy: 0.9952 - weighted_accuracy: 0.9952 - val_loss: 2.7662 - val_accuracy: 0.7133 - val_weighted_accuracy: 0.7133
Epoch 83/100
66/66 [==============================] - 0s 6ms/step - loss: 0.0228 - accuracy: 0.9952 - weighted_accuracy: 0.9952 - val_loss: 2.9545 - val_accuracy: 0.6967 - val_weighted_accuracy: 0.6967
Epoch 84/100
66/66 [==============================] - 0s 6ms/step - loss: 0.0165 - accuracy: 0.9971 - weighted_accuracy: 0.9971 - val_loss: 2.9412 - val_accuracy: 0.6989 - val_weighted_accuracy: 0.6989
Epoch 85/100
66/66 [==============================] - 0s 6ms/step - loss: 0.0175 - accuracy: 0.9938 - weighted_accuracy: 0.9938 - val_loss: 2.9271 - val_accuracy: 0.7033 - val_weighted_accuracy: 0.7033
Epoch 86/100
66/66 [==============================] - 0s 7ms/step - loss: 0.0186 - accuracy: 0.9943 - weighted_accuracy: 0.9943 - val_loss: 3.0089 - val_accuracy: 0.7111 - val_weighted_accurac

88/88 [==============================] - 1s 7ms/step - loss: 0.1277 - accuracy: 0.9593 - weighted_accuracy: 0.9593 - val_loss: 1.7256 - val_accuracy: 0.6975 - val_weighted_accuracy: 0.6975
Epoch 23/100
88/88 [==============================] - 1s 7ms/step - loss: 0.1052 - accuracy: 0.9643 - weighted_accuracy: 0.9643 - val_loss: 1.7873 - val_accuracy: 0.7000 - val_weighted_accuracy: 0.7000
Epoch 24/100
88/88 [==============================] - 1s 7ms/step - loss: 0.0921 - accuracy: 0.9718 - weighted_accuracy: 0.9718 - val_loss: 1.6580 - val_accuracy: 0.6933 - val_weighted_accuracy: 0.6933
Epoch 25/100
88/88 [==============================] - 1s 7ms/step - loss: 0.0834 - accuracy: 0.9764 - weighted_accuracy: 0.9764 - val_loss: 1.7779 - val_accuracy: 0.7133 - val_weighted_accuracy: 0.7133
Epoch 26/100
88/88 [==============================] - 1s 7ms/step - loss: 0.0690 - accuracy: 0.9768 - weighted_accuracy: 0.9768 - val_loss: 1.8698 - val_accuracy: 0.6833 - val_weighted_accuracy: 0.6833
Epo

Epoch 63/100
88/88 [==============================] - 1s 6ms/step - loss: 0.0228 - accuracy: 0.9932 - weighted_accuracy: 0.9932 - val_loss: 2.8790 - val_accuracy: 0.6925 - val_weighted_accuracy: 0.6925
Epoch 64/100
88/88 [==============================] - 1s 6ms/step - loss: 0.0162 - accuracy: 0.9946 - weighted_accuracy: 0.9946 - val_loss: 2.9399 - val_accuracy: 0.6917 - val_weighted_accuracy: 0.6917
Epoch 65/100
88/88 [==============================] - 1s 7ms/step - loss: 0.0201 - accuracy: 0.9957 - weighted_accuracy: 0.9957 - val_loss: 2.9767 - val_accuracy: 0.7067 - val_weighted_accuracy: 0.7067
Epoch 66/100
88/88 [==============================] - 1s 7ms/step - loss: 0.0326 - accuracy: 0.9932 - weighted_accuracy: 0.9932 - val_loss: 3.0746 - val_accuracy: 0.6842 - val_weighted_accuracy: 0.6842
Epoch 67/100
88/88 [==============================] - 1s 7ms/step - loss: 0.0246 - accuracy: 0.9929 - weighted_accuracy: 0.9929 - val_loss: 2.9696 - val_accuracy: 0.6892 - val_weighted_accurac

...

114/114 [==============================] - 1s 7ms/step - loss: 0.5451 - accuracy: 0.8008 - weighted_accuracy: 0.8008 - val_loss: 0.7364 - val_accuracy: 0.7353 - val_weighted_accuracy: 0.7353
Epoch 9/100
114/114 [==============================] - 1s 7ms/step - loss: 0.5097 - accuracy: 0.8077 - weighted_accuracy: 0.8077 - val_loss: 0.7233 - val_accuracy: 0.7500 - val_weighted_accuracy: 0.7500
Epoch 10/100
114/114 [==============================] - 1s 7ms/step - loss: 0.4633 - accuracy: 0.8302 - weighted_accuracy: 0.8302 - val_loss: 0.7861 - val_accuracy: 0.7468 - val_weighted_accuracy: 0.7468
Epoch 11/100
114/114 [==============================] - 1s 7ms/step - loss: 0.4330 - accuracy: 0.8429 - weighted_accuracy: 0.8429 - val_loss: 0.8346 - val_accuracy: 0.7256 - val_weighted_accuracy: 0.7256
Epoch 12/100
114/114 [==============================] - 1s 7ms/step - loss: 0.3821 - accuracy: 0.8549 - weighted_accuracy: 0.8549 - val_loss: 0.9095 - val_accuracy: 0.7212 - val_weighted_accuracy: 0

114/114 [==============================] - 1s 7ms/step - loss: 0.0340 - accuracy: 0.9904 - weighted_accuracy: 0.9904 - val_loss: 2.5728 - val_accuracy: 0.7019 - val_weighted_accuracy: 0.7019
Epoch 49/100
114/114 [==============================] - 1s 7ms/step - loss: 0.0335 - accuracy: 0.9915 - weighted_accuracy: 0.9915 - val_loss: 2.5001 - val_accuracy: 0.6981 - val_weighted_accuracy: 0.6981
Epoch 50/100
114/114 [==============================] - 1s 7ms/step - loss: 0.0443 - accuracy: 0.9904 - weighted_accuracy: 0.9904 - val_loss: 2.6352 - val_accuracy: 0.6942 - val_weighted_accuracy: 0.6942
Epoch 51/100
114/114 [==============================] - 1s 7ms/step - loss: 0.0334 - accuracy: 0.9907 - weighted_accuracy: 0.9907 - val_loss: 2.4393 - val_accuracy: 0.7045 - val_weighted_accuracy: 0.7045
Epoch 52/100
114/114 [==============================] - 1s 7ms/step - loss: 0.0237 - accuracy: 0.9937 - weighted_accuracy: 0.9937 - val_loss: 2.5433 - val_accuracy: 0.6955 - val_weighted_accuracy: 

114/114 [==============================] - 1s 7ms/step - loss: 0.0128 - accuracy: 0.9953 - weighted_accuracy: 0.9953 - val_loss: 3.4221 - val_accuracy: 0.6962 - val_weighted_accuracy: 0.6962
Epoch 89/100
114/114 [==============================] - 1s 7ms/step - loss: 0.0156 - accuracy: 0.9956 - weighted_accuracy: 0.9956 - val_loss: 3.2792 - val_accuracy: 0.6885 - val_weighted_accuracy: 0.6885
Epoch 90/100
114/114 [==============================] - 1s 7ms/step - loss: 0.0323 - accuracy: 0.9918 - weighted_accuracy: 0.9918 - val_loss: 3.3106 - val_accuracy: 0.6840 - val_weighted_accuracy: 0.6840
Epoch 91/100
114/114 [==============================] - 1s 8ms/step - loss: 0.0441 - accuracy: 0.9868 - weighted_accuracy: 0.9868 - val_loss: 2.7859 - val_accuracy: 0.7032 - val_weighted_accuracy: 0.7032
Epoch 92/100
114/114 [==============================] - 1s 7ms/step - loss: 0.0320 - accuracy: 0.9885 - weighted_accuracy: 0.9885 - val_loss: 2.7117 - val_accuracy: 0.6949 - val_weighted_accuracy: 

105/105 [==============================] - 1s 6ms/step - loss: 0.0986 - accuracy: 0.9679 - weighted_accuracy: 0.9679 - val_loss: 2.0145 - val_accuracy: 0.7097 - val_weighted_accuracy: 0.7097
Epoch 29/100
105/105 [==============================] - 1s 6ms/step - loss: 0.0875 - accuracy: 0.9720 - weighted_accuracy: 0.9720 - val_loss: 1.9811 - val_accuracy: 0.7118 - val_weighted_accuracy: 0.7118
Epoch 30/100
105/105 [==============================] - 1s 6ms/step - loss: 0.0555 - accuracy: 0.9833 - weighted_accuracy: 0.9833 - val_loss: 2.1242 - val_accuracy: 0.7035 - val_weighted_accuracy: 0.7035
Epoch 31/100
105/105 [==============================] - 1s 6ms/step - loss: 0.0405 - accuracy: 0.9866 - weighted_accuracy: 0.9866 - val_loss: 2.3402 - val_accuracy: 0.7000 - val_weighted_accuracy: 0.7000
Epoch 32/100
105/105 [==============================] - 1s 6ms/step - loss: 0.0349 - accuracy: 0.9884 - weighted_accuracy: 0.9884 - val_loss: 2.4179 - val_accuracy: 0.7007 - val_weighted_accuracy: 

105/105 [==============================] - 1s 6ms/step - loss: 0.0331 - accuracy: 0.9911 - weighted_accuracy: 0.9911 - val_loss: 2.7220 - val_accuracy: 0.6965 - val_weighted_accuracy: 0.6965
Epoch 69/100
105/105 [==============================] - 1s 6ms/step - loss: 0.0297 - accuracy: 0.9914 - weighted_accuracy: 0.9914 - val_loss: 2.8727 - val_accuracy: 0.7097 - val_weighted_accuracy: 0.7097
Epoch 70/100
105/105 [==============================] - 1s 6ms/step - loss: 0.0341 - accuracy: 0.9908 - weighted_accuracy: 0.9908 - val_loss: 2.7360 - val_accuracy: 0.6993 - val_weighted_accuracy: 0.6993
Epoch 71/100
105/105 [==============================] - 1s 6ms/step - loss: 0.0294 - accuracy: 0.9902 - weighted_accuracy: 0.9902 - val_loss: 2.8200 - val_accuracy: 0.6854 - val_weighted_accuracy: 0.6854
Epoch 72/100
105/105 [==============================] - 1s 6ms/step - loss: 0.0301 - accuracy: 0.9914 - weighted_accuracy: 0.9914 - val_loss: 2.7692 - val_accuracy: 0.7042 - val_weighted_accuracy: 

In [33]:
df_precision = pd.DataFrame(precision_all,columns =['tr_avg_pre','TrP1', 'TrP2','TrP3','TrP4','TrP5','ts_avg_pre','TsP1','TsP2','TsP3','TsP4','TsP5'] )
df_precision.to_csv(path+'precision_LSTMCNN.csv')